In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mnist import MNIST

In [4]:
# Reading the dataset
mndata = MNIST('samples/')

X_train, y_train = mndata.load_training()
X_test, y_test = mndata.load_testing()

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [5]:
class Layer_Dense():
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
        self.inputs = inputs # we want to remember what our inputs are

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        # for easier understanding, we can calculate dbiases with
        # np.dot(dvalues, np.array([1, 1, 1]).T) (second argument is derivative of z with respect to biases for all samples)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

In [6]:
class Activation_ReLU():
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
        self.inputs = inputs

    def backward(self, dvalues):
        """
        the same but with multiplication and more clearly:

        drelu_dinputs[inputs <= 0] = 0
        drelu_dinputs[inputs > 0] = 1
        dinputs = dvalues * drelu_dinputs
        """
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax():
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / exp_values.sum(axis=1, keepdims=True)
        self.output = probabilities

        # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)
        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output and
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)

            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [7]:
class Loss():
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true

        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)

        return -np.log(correct_confidences)

    def backward(self, dvalues, y_true):
        # dvalues actually is a matrix which contains predictions (samples x predictions)
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = dvalues[0]
        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        dinputs = - y_true / dvalues

        # this normalization step ensures that the gradients are averaged across all samples
        dinputs = dinputs / samples

class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

        # Backward pass
    def backward(self, dvalues, y_true):
        # derivative of softmax + loss is (y_hat - y_true)
        # Number of samples
        samples = len(dvalues)

        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [8]:
class Optimizer_SGD():
    def __init__(self, learning_rate=1.0, decay=.0, momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.momentum = momentum
        self.iterations = 0

    # function for updating learning rate
    def pre_update_params(self):
        if self.decay:
            # every time we multiply our learning rate (for example 1) with smaller and smaller float (<=1)
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        # if we use momentum
        if self.momentum:
            # If layer does not contain momentum arrays, create them
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            # while updating our wwights we take into count our prevous gradients
            # momentum walue determines how much we want to keep out prevous gradients
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates

        else: # If we dont use momentum
            weight_updates = -self.learning_rate * layer.dweights
            bias_updates = -self.learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1

In [9]:
class Optimizer_AdaGrad():
    def __init__(self, learning_rate=1.0, decay=.0, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    # function for updating learning rate
    def pre_update_params(self):
        if self.decay:
            # every time we multiply our learning rate (for example 1) with smaller and smaller float (<=1)
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        # if we use momentum
        if not hasattr(layer, 'weight_chache'):
            layer.weight_chache = np.zeros_like(layer.weights)
            layer.bias_chache = np.zeros_like(layer.biases)

        layer.weight_chache += layer.dweights ** 2
        layer.bias_chache += layer.dbiases ** 2

        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_chache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_chache) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

In [10]:
class Optimizer_RMSprop():
    def __init__(self, learning_rate=0.001, decay=0, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    # function for updating learning rate
    def pre_update_params(self):
        if self.decay:
            # every time we multiply our learning rate (for example 1) with smaller and smaller float (<=1)
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        # if we use momentum
        if not hasattr(layer, 'weight_chache'):
            layer.weight_chache = np.zeros_like(layer.weights)
            layer.bias_chache = np.zeros_like(layer.biases)

        # layer.weight_chache = self.rho * layer.weight_chache + (1 - self.rho) * layer.dweights ** 2
        # layer.bias_chache = self.rho * layer.bias_chache + (1 - self.rho) * layer.dbiases ** 2

        layer.weight_chache = self.rho * layer.weight_chache + \
                            (1 - self.rho) * layer.dweights**2
        layer.bias_chache = self.rho * layer.bias_chache + \
                            (1 - self.rho) * layer.dbiases**2

        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_chache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_chache) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

In [11]:
class Optimizer_Adam():
    def __init__(self, learning_rate=0.001, decay=0, epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    # function for updating learning rate
    def pre_update_params(self):
        if self.decay:
            # every time we multiply our learning rate (for example 1) with smaller and smaller float (<=1)
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        # if we use momentum
        if not hasattr(layer, 'weight_chache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_chache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_chache = np.zeros_like(layer.biases)


        # update momentums
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        # correct momentums
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1**(self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1**(self.iterations + 1))

        # update chache
        layer.weight_chache = self.beta_2 * layer.weight_chache + (1 - self.beta_2) * layer.dweights**2
        layer.bias_chache = self.beta_2 * layer.bias_chache + (1 - self.beta_2) * layer.dbiases**2

        # correct chache
        weight_chache_corrected = layer.weight_chache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_chache_corrected = layer.bias_chache / (1 - self.beta_2 ** (self.iterations + 1))


        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_chache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_chache_corrected) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

In [12]:
# Create model structure
dense1 = Layer_Dense(784, 10)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(10, 10)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

optimizer = Optimizer_Adam(decay=5e-5)

In [13]:
for epoch in range(200):
    # Forward pass
    dense1.forward(X_train)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, y_train)

    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y_train.shape) == 2:
        y_train = np.argmax(y_train, axis=1)

    accuracy = np.mean(predictions==y_train)

    # if epoch % 100 == 0:
    print(f'epoch: {epoch}, ' + f'acc: {accuracy:.3f}, ' + f'loss: {loss:.3f}, ' + f'lr: {optimizer.current_learning_rate}')

    # Backward pass
    loss_activation.backward(loss_activation.output, y_train)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Optimizing
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.107, loss: 2.463, lr: 0.001
epoch: 1, acc: 0.191, loss: 2.195, lr: 0.001
epoch: 2, acc: 0.261, loss: 2.061, lr: 0.000999950002499875
epoch: 3, acc: 0.294, loss: 1.958, lr: 0.000999900009999
epoch: 4, acc: 0.316, loss: 1.871, lr: 0.0009998500224966255
epoch: 5, acc: 0.358, loss: 1.778, lr: 0.0009998000399920016
epoch: 6, acc: 0.429, loss: 1.677, lr: 0.0009997500624843788
epoch: 7, acc: 0.488, loss: 1.571, lr: 0.000999700089973008
epoch: 8, acc: 0.521, loss: 1.474, lr: 0.0009996501224571398
epoch: 9, acc: 0.533, loss: 1.394, lr: 0.0009996001599360256
epoch: 10, acc: 0.544, loss: 1.332, lr: 0.0009995502024089159
epoch: 11, acc: 0.558, loss: 1.275, lr: 0.0009995002498750624
epoch: 12, acc: 0.576, loss: 1.212, lr: 0.0009994503023337165
epoch: 13, acc: 0.598, loss: 1.148, lr: 0.0009994003597841295
epoch: 14, acc: 0.623, loss: 1.093, lr: 0.0009993504222255533
epoch: 15, acc: 0.643, loss: 1.044, lr: 0.0009993004896572402
epoch: 16, acc: 0.662, loss: 0.998, lr: 0.00099925056207